In [1]:

from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(
    os.environ.get('WORK_DIR')) #type: ignore
sys.path.append(
    os.environ.get('DBASE_DIR')) #type: ignore
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from EventDriven.data import HistoricTradeDataHandler
from EventDriven.event import *
from queue import Queue
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import BBandsTrend2
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from EventDriven.backtest import OptionSignalBacktest
from copy import deepcopy
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-03-13 10:33:54 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [2]:
thetadata_start = '2021-01-01'

In [3]:
import json
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
key = 10
with open(f'../input/profitable_weights_{key}.json', 'r') as f:
    weights = json.load(f)
ttrades__ = pd.read_csv(f'../input/profitable_trades_{key}.csv').iloc[:, 1:]
ttrades__ = ttrades__[ttrades__.Ticker == 'NVDA']
ttrades__

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
4,151.0,518.0,753.0,18.979196,49.243999,4569.985385,1.59463,2023-01-25,2024-01-02,342 days,NVDA


In [4]:
trades_ = ttrades__.copy()
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
4,151.0,518.0,753.0,18.979196,49.243999,4569.985385,1.59463,2023-01-25,2024-01-02,342 days,NVDA


In [5]:
symbol_list = trades_.Ticker.unique()
untraded_symbols = [s for s in weights.keys() if s not in trades_.Ticker.unique()]
for s in untraded_symbols:
    weights.pop(s)
pd.Series(weights).sort_values(ascending=False)

NVDA    0.062106
dtype: float64

In [6]:

from EventDriven.riskmanager import RiskManager, close_cache, spot_cache, chain_cache, oi_cache, LOOKBACKS, order_cache
from pandas.tseries.offsets import BDay

rm = RiskManager(None, None, 1000000)
rm.OrderPicker.liquidity_threshold = 2
rm.OrderPicker.lookback = 10
rm.OrderPicker.data_availability_threshold = 0.15
date, tick = '2023-07-05', 'AVGO'
date, tick = '2024-08-13', 'TSM'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'
order_settings = {'type': 'spread',
 'specifics': [{'direction': 'long',
   'rel_strike': .50,
   'dte': 210,
   'moneyness_width': 0.5},
  #  {'direction': 'short',
  # 'rel_strike': .60,
  # 'dte': 270,
  # 'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}



order =  rm.OrderPicker.get_order(tick, date, right, 2, order_settings)
order

Saving to cache from db
Updating db rates data


{'result': 'SUCCESSFUL',
 'data': {'long': ['TSM20250321C280'],
  'trade_id': '&L:TSM20250321C280',
  'close': 1.515}}

In [7]:
max_cash = {}
cash = 20_000
for s, w in weights.items():
    if w * cash > 200:
        max_cash[s] = 2
    elif w * cash > 100:
        max_cash[s] = 1
    else:
        max_cash[s] = 0.5
max_cash

{'NVDA': 2}

In [121]:
#Backtest class 
## Find a way to not always reinitialize the backtest class, when want to redo
from EventDriven.execution import exec_cache
evb_backtest = OptionSignalBacktest(trades_, initial_capital=cash)

In [122]:
evb_backtest.portfolio.initial_capital

20000.0

In [123]:
w_map = {x: w  * 0.75 for x, w in weights.items()} ## 75% of the weights for each stock
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.risk_manager.OrderPicker.liquidity_threshold = 100
evb_backtest.portfolio.risk_manager.OrderPicker.lookback = 10
evb_backtest.portfolio.risk_manager.OrderPicker.data_availability_threshold = 0.5
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .850,
   'dte': 300,
   'moneyness_width': 0.35},
   {'direction': 'short',
  'rel_strike': .60,
  'dte': 300,
  'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}

# evb_backtest.portfolio.order_settings = {'type': 'naked',
#  'specifics': [{'direction': 'long',
#    'rel_strike': .750,
#    'dte': 210,
#    'moneyness_width': 0.5},

# ],
#  'name': 'vertical_spread'}


evb_backtest.portfolio.max_contract_price = max_cash
evb_backtest.executor.commission_rate = 0.65/100
evb_backtest.executor.commission_rate
evb_backtest.executor.max_slippage_pct = 0.075
evb_backtest.portfolio.roll_map = 180
evb_backtest.portfolio.allocated_cash_map
evb_backtest.portfolio.max_contract_price
evb_backtest.portfolio.roll_map

{'NVDA': 180}

In [124]:
signals = evb_backtest.bars.signal_df
# series = signals.set_index('Date')['MSFT']
# series[series != 0]
signals

,Date,NVDA
0,2023-01-25,1
1,2023-01-26,0
2,2023-01-27,0
3,2023-01-30,0
4,2023-01-31,0
5,2023-02-01,0
6,2023-02-02,0
7,2023-02-03,0
8,2023-02-06,0
9,2023-02-07,0


In [125]:
signals_df = deepcopy(signals).set_index('Date')
signals_df[signals_df!=-1].sum().sum()

1.0

In [126]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

Processing event: MARKET
Processing event: SIGNAL
Processing event: ORDER
Processing event: FILL
2023-01-25 00:00:00
NVDA BUY
Event queue is empty, processed 4 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing even

In [131]:
idx = 0
buys = [x  for x in exec_cache['fill'].values() if x.direction == 'BUY']
buy1 = buys[idx]
print("Buy")
print(buy1.datetime,buy1.fill_cost/buy1.quantity, buy1.market_value)

sells = [x  for x in exec_cache['fill'].values() if x.direction == 'SELL']
sell1 = sells[idx]
print("Sell")
print(sell1.datetime,sell1.fill_cost/sell1.quantity, sell1.market_value)



Buy
2023-01-25 00:00:00 2.0292157574246827 8.064863029698731
Sell
2024-01-02 00:00:00 1.0975554369362588 97.72887845039078


In [130]:
sells

In [92]:
trades = evb_backtest.portfolio._trades.copy()#
trades[['Positions','Ticker','PnL', 'ReturnPct', 'EntryPrice', 'ExitPrice', 'EntryMarketValue', 'ExitMarketValue', 'EntryTime', 'ExitTime', 'EntryPrice', 'ExitPrice', 'Quantity', 'SignalID']]
# len(trades['SignalID'].unique())

,Positions,Ticker,PnL,ReturnPct,EntryPrice,ExitPrice,EntryMarketValue,ExitMarketValue,EntryTime,ExitTime,EntryPrice,ExitPrice,Quantity,SignalID
0,&L:NVDA20240119C380&S:NVDA20240119C450,NVDA,13750.634591,16.996673,202.254797,3639.913444,803.819187,14564.853777,2023-01-25,2023-07-24,202.254797,3639.913444,4,NVDA20230125LONG
1,&L:NVDA20240621C570&S:NVDA20240621C575,NVDA,-2451.801840,-0.175195,168.610809,139.071028,13886.797152,11650.795312,2023-07-25,2023-12-26,168.610809,139.071028,83,NVDA20230125LONG
2,&L:NVDA20240920C780&S:NVDA20240920C800,NVDA,NaN,NaN,153.767048,NaN,609.868193,NaN,2023-12-26,NaT,153.767048,NaN,4,NVDA20230125LONG
3,&L:NVDA20240920C650&S:NVDA20240920C660,NVDA,-4345.197858,-0.196255,205.005518,164.772205,22000.195947,17935.798089,2023-12-27,2024-01-02,205.005518,164.772205,108,NVDA20230125LONG


In [91]:
evb_backtest.portfolio.plot_portfolio()

In [18]:
evb_backtest.portfolio.aggregate()

Start                                                    2023-01-25 00:00:00
End                                                      2024-01-02 00:00:00
Duration                                                   342 days 00:00:00
Exposure Time [%]                                                      100.0
Equity Final [$]                                                    37313.79
Equity Peak [$]                                                 41866.273056
Return [%]                                                         86.842826
Buy & Hold Return [%]                                             146.178132
CAGR [%]                                                           94.864897
Volatility Ann. [%]                                                69.982562
Sharpe Ratio                                                        1.285276
Sortino Ratio                                                       2.003141
Skew                                                                 2.27629

In [19]:
eq = evb_backtest.portfolio._equity.copy()
eq[eq.index >= '2023-07-20']

,NVDA,cash,commission,Total
datetime,,,,
2023-07-20,15892.276354,19068.409277,0.052,34960.685631
2023-07-21,15212.276354,19068.409277,0.052,34280.685631
2023-07-24,14512.266372,19068.409277,0.104,33580.675649
2023-07-25,14162.609947,19068.409277,1.170,33231.019223
2023-07-26,13752.609947,19068.409277,1.170,32821.019223
...,...,...,...,...
2023-12-27,21155.335090,19068.409277,4.771,40223.744367
2023-12-28,21420.335090,19068.409277,4.771,40488.744367
2023-12-29,21685.335090,19068.409277,4.771,40753.744367


In [20]:
#0	&L:NVDA20240119C380&S:NVDA20240119C450
price = (evb_backtest.portfolio.options_data['NVDA20240119C380'] - evb_backtest.portfolio.options_data['NVDA20240119C450']).copy()
price[price.index >= '2023-01-25']
price[price.index.isin( ['2023-01-25', '2023-07-24'])]

,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,
2023-01-25,1.69,2.04,1.67,2.04,-5,-253,1.9,59,1.95,1.925,2.082202
2023-07-24,35.90,37.75,40.20,37.86,-183,127,36.7,94,39.80,38.250,38.037282


In [21]:

pd.set_option('display.max_rows', 10000)
evb_backtest.portfolio.get_all_positions()

,,long,short,trade_id,close,quantity,market_value
datetime,symbol,,,,,,
2023-01-25,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,770.0,4,770.0
2023-01-26,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,844.0,4,844.0
2023-01-27,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,976.0,4,976.0
2023-01-30,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,688.0,4,688.0
2023-01-31,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,730.0,4,730.0
2023-02-01,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,956.0,4,956.0
2023-02-02,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,1358.0,4,1358.0
2023-02-03,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,1244.0,4,1244.0
2023-02-06,NVDA,[NVDA20240119C380],[NVDA20240119C450],&L:NVDA20240119C380&S:NVDA20240119C450,1218.0,4,1218.0


In [22]:


""""
Why do these two return weird stuff after run?
    current_weighted_holdings
    current_positions

I can't reconcile the cost with the data (NVM, haha)

"""
# evb_backtest.portfolio.all_positions

'"\nWhy do these two return weird stuff after run?\n    current_weighted_holdings\n    current_positions\n\nI can\'t reconcile the cost with the data (NVM, haha)\n\n'

In [23]:
order_cache['2024-01-03']

KeyError: '2024-01-03'

## Extend for  get_port_stats
- Buy & Hold
- Dates
- Trades
- _strategy in Aggregate
- The function

In [214]:
evb_backtest.trades.to_csv(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/output/profitable_trades_options_{key}.csv')